In [ ]:
%pip install Sentence-transformers
%pip install re
%pip install nltk

In [139]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

## Data Preprocessing 
  ### 1. Removal of Stopwords
  ### 2. Lemetization
  ### 3. Removing any non-meaningful words with the help of re module.
  
#### Note: Stemming was not performed as it was observed that it was mostly damaging the information which can be used to find similarity between the sentences.

In [140]:
nltk.download('punkt')
nltk.download('wordnet')


PATH='Collection//euro_news//para.csv'
df=pd.read_csv(PATH,sep='|',names=['Channel','Date','Time','Place','News'])

[nltk_data] Error loading punkt: <urlopen error [Errno 104] Connection
[nltk_data]     reset by peer>
[nltk_data] Downloading package wordnet to /home/udayb/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [141]:
stop_words=set(stopwords.words('english'))

# Creating  the object for lemmatisation
lemt = WordNetLemmatizer()


for ind, row  in df.iterrows():

	text=df.loc[(ind,'News')]
	words=nltk.word_tokenize(	text	)
	
	NEW_HDL=""
	for word in words:
		if word not in stop_words:
			NEW_HDL=NEW_HDL+' '+lemt.lemmatize(word)

	# Removing strings which are inside the parantheses from the news headline 
	NEW_HDL=re.sub('\[[a-zA-Z0-9,\' ]+\]',' ',NEW_HDL)
	
	df.loc[(ind,'News')]=NEW_HDL
	
df

,Channel,Date,Time,Place,News
0,Euro News,01/01/23,Evening,Outside India,10 year joining European Union Croatia swit...
1,Euro News,01/01/23,Midday,Outside India,half hour 2023 Air Raid Sirens ring across Uk...
2,Euro News,01/01/23,Morning,Outside India,Noble kind word used Pope Francis describe pr...
3,Euro News,01/02/23,Evening,Outside India,sign slight easing cost living crisis inflati...
4,Euro News,01/02/23,Midday,Outside India,foreign half million worker expected strike U...
...,...,...,...,...,...
1531,Euro News,31/10/23,Midday,Outside India,Israeli air strike continue dark hundred thou...
1532,Euro News,31/10/23,Morning,Outside India,defiant Benjamin Netanyahu ruled ceasefire Ga...
1533,Euro News,31/12/22,Evening,Outside India,European leader commemorate former Pope Ben...
1534,Euro News,31/12/22,Midday,Outside India,former Pope Benedict XVI first pontiff 600 ...


## Now, we proceed towards finding similarity.

1. __Making a function which commutes the Cosine Similarity__

In [151]:
from sentence_transformers import SentenceTransformer as sen_tr, util

def EMBED(news_list:dict):
  """
    This gives an nd array of Embeddings in the format of:
    [  Morning , Midday , Evening ]
  """

  # Creating an object for the model
  model = sen_tr('all-MiniLM-L6-v2')
  
  # Creating Embeddings for different timings of the day
  emb0 = model.encode(news_list['Morning'])
  emb1 = model.encode(news_list['Midday'])
  emb2 = model.encode(news_list['Evening'])
  
  return np.array([ emb0 ,emb1 , emb2 ])

def COS_SIM(embed: dict):
  """
    This function returns the cosine-similarity of different timings in the format:

  """
  def rel(val):
    return val*100
  
  ans={'Morning-Midday':0,'Midday-Evening':0,'Morning-Evening':0}

  ans['Morning-Midday'] = rel(  util.cos_sim(embed[0],embed[1]  )[0,0].item())
  ans['Morning-Evening'] = rel(  util.cos_sim(embed[0],embed[2]  )[0,0].item())
  ans['Midday-Evening'] = rel(  util.cos_sim(embed[1],embed[2]  )[0,0].item())

  return ans

In [152]:
un_date=df['Date'].unique()

data_c={'Date':[],'Morning-Midday':[],'Midday-Evening':[],'Morning-Evening':[]}


for DATE in un_date:


  df_day=df[df['Date']==DATE]

  news={
          'Morning':  '',
          'Midday' :  '',
          'Evening':  ''
        }

  for ind, row in df_day.iterrows():
    news[row['Time']]=row['News']
  
  
  embed = EMBED(news)
  dt  = COS_SIM(embed)
  data_c['Date'].append(DATE);  data_c['Morning-Midday'].append(dt['Morning-Midday']);  data_c['Midday-Evening'].append(dt['Midday-Evening']);  data_c['Morning-Evening'].append(dt['Morning-Evening'])


In [153]:
per_df1=pd.DataFrame(data_c)
per_df1['Morning-Evening'].min()

2.7643445879220963

In [154]:
per_df1

,Date,Morning-Midday,Midday-Evening,Morning-Evening
0,01/01/23,73.351413,44.613236,34.173316
1,01/02/23,57.157195,59.162945,42.400160
2,01/03/23,73.402280,48.875538,48.308423
3,01/04/22,80.984455,67.516607,70.125920
4,01/04/23,1.809320,62.498868,2.764345
...,...,...,...,...
507,31/08/22,56.430906,84.331512,63.496947
508,31/08/23,45.474049,64.334393,57.299179
509,31/10/22,96.402335,84.863830,83.515412
510,31/10/23,86.921757,95.947510,85.745031
